In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Distgen basic usage
--------------------

Distgen supports nearly arbitrary generation of 6D + time particle coordinates.  

# Generator

Generator class defines the main object that creates a beam distribution.  Typical usage is to supply the Generator.\_\_init\_\_(input,verbose) with an input file name and a verbose boolean value to control the level of output printed to the user.  Input files can be either json or YAML.  To produce a beam distribution, call Generator.beam().  

In [2]:
# Basic Example using a json file:
input_file = "data/rad.gaussian.in.json"
from distgen import Generator

In [7]:
# Create a generator object
gen = Generator(input_file, verbose=0) 

Printing a Generator object displays the input data read from a distgen input file:

In [8]:
print(gen)

<disgten.Generator with input: 
beam:
  params:
    MTE:
      units: meV
      value: 150
    total_charge:
      units: pC
      value: 10
  particle_count: 200000
  rand_type: hammersley
  start_type: cathode
output:
  file: rad.gaussian.out.txt
  type: gpt
r_dist:
  params:
    sigma_xy:
      units: mm
      value: 2
  type: radial_gaussian
t_dist:
  params:
    max_t:
      units: ps
      value: 2
    min_t:
      units: ps
      value: -2
  type: uniform

>


The input structure allows for four top levels: beam, var_dists, transforms, and output.  Any other input will throw an exception when attempting to create the beam:

In [10]:
try:
    gen.input['foo']='bar'
    gen.beam()   
except Exception as ex:
    print(ex)

Unexpected distgen input parameter: foo


# Beam Object

The particle coordinates created by the Generator class are stored in a beam obect.  The main underlying coordinates are the 6D phase space coordinates $x$, $y$, $z$, $p_x$, $p_y$, $p_z$, and time $t$.  These can be accessed via the [...] operator.  The bunch charge is given by beam.q.

In [12]:
gen = Generator(input_file, verbose=0) 
beam = gen.beam()
print("X coordinates:", beam["x"])
print("Bunch charge:",beam.q)

X coordinates: [0.0007186821990022249 0.0025719423335885303 0.0017315595335433654 ... 0.002841790247471585 0.001973987431900489 0.004164315402213629] meter
Bunch charge: 10 picocoulomb


Currently, the code assumes one particle species (default: electrons) per beam object.  The bunch contains an array of particle weights (currently uniform) for use with averaging.  The particle weights $w$ are normalized $\sum_i{w_i}=1$ and are used for computing expectation values over the particle ensemble. 

Simple examples include beam.avg: $\langle\mathcal{O}\rangle = \sum_i w_i\mathcal{O}_i$ and beam.std(): $\sigma_{\mathcal{O}} = \sqrt{\sum_i{w_i(\mathcal{O}_i-\langle\mathcal{O}\rangle)^2}}$.

Other examples include the normalized and geometric emittance and the twiss parameters $\beta$ and $\alpha$  (See the distgen.beam.py).



In [ ]:
import numpy as np
print("Beam species:",beam.species)
print("Weights:",beam["w"])
print("Normalization sum(weights): ",np.sum(beam["w"]))
print('Avgerage of X: {:0.3f~P}'.format(beam.avg("x")))
print('Standard Deviation of X: {:0.3f~P}'.format(beam.std("x",desired_units="mm")))

The beam object supports a limited number of input params: total_charge, MTE (for cathode start), tstart (for tstart), and specieis (CURRENTLY DEFAULTS TO ELECTRONS).  total_charge is required:

In [14]:
gen = Generator(input_file, verbose=0) 
gen.input['beam']['params']={'MTE': {'value': 150, 'units': 'meV'}}
try:
    gen.beam()   
except Exception as ex:
    print(ex)

Required input parameter total_charge to Beam.__init__(**kwargs) was not found.


In [ ]:
Unexpected inputs will cause an exception:

In [15]:
gen = Generator(input_file, verbose=0) 
gen.input['beam']['params']['foo']='bar'
try:
    gen.beam()   
except Exception as ex:
    print(ex)

Incorrect param given to Beam.__init__(**kwargs): foo
Allowed params: ['MTE', 'tstart', 'species', 'total_charge', 'verbose']


# Distribution Objects
Distgen handles distirbutions using classes controlled by Generator.  Currently there are three types supported: 1D distributions (Dist1d), radial distributions for the $r$ coordinate, and 2D distributions.  Each distribution type keeps an internal list of the required and optional parameters that must be passed to the object.  When a distribution object is initialized, the input parameters are checked against this list. Unexpected inputs will throw exceptions:

In [13]:
gen = Generator(input_file, verbose=0) 
try:
    gen.input['r_dist']['params']['foo']='bar'
    gen.beam()   
except Exception as ex:
    print(ex)

Incorrect param given to NormRad.__init__(**kwargs): foo
Allowed params: ['sigma_xy', 'truncation_fraction', 'truncation_radius_left', 'truncation_radius_right', 'n_sigma_cutoff_left', 'n_sigma_cutoff_left', 'n_sigma_cutoff', 'truncation_radius', 'truncation_radius_left', 'truncation_radius_right', 'verbose']


As will missing required parameters:

In [ ]:
gen = Generator('data/beer.can.in.json', verbose=0) 
try:
    gen.input['r_dist']['params']={}
    gen.beam()   
except Exception as ex:
    print(ex)

# Output writer functionality 
Distgen supports writer functions for various output formats/codes.  Currently these include GPT and ASTRA, and the openPMD secification.  To write to a file, use distgen.writers.writer with appropriate code type specified as well as the beam object and desired output file.  The writer function accepts additional parameters required for any code dependent output.

In [ ]:
#Distgen supports writer functions for various output formats/codes (GPT,ASTRA)
from distgen.writers import *
gen = Generator('data/beer.can.in.json', verbose=0) 
p = gen.params

# Typically the user can just call the general write function and specify the desired format:
writer("gpt", beam, "rad.gaussian.gpt.out.txt",verbose=1,params=p)
writer("astra", beam, "rad.gaussian.astra.out.txt",verbose=1,params=p)
writer("openPMD", beam, "rad.gaussian.openPMD.out.h5",verbose=1)

# Plotting 
Distgen provides the user with some basic plotting routines for help in visualizing the beam coordinates.  The most common plot types include plot_2d, plot_1d, and plot_current_profile.  Below shows an example of how to plot 2 coordinates, in this case $x$ and $y$.  The coloring scheme is a scatter histogram.

In [ ]:
# The distgen module has some basic plotting routines
from distgen.plot import *

units = {"x":"mm","y":"mm",'z':'mm',"px":"keV/c","py":"keV/c","pz":'keV/c',"t":"ps","q":"pC",'thetax':'mrad'}

# plot_2d allows for plotting a 2D subspace of the beam coordinates:

fig=plt.figure(1)
# X-Y Plot
plot_2d(beam,1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=100,axis="equal",title_on=True);

plot_current_profiles is a special case of the plot_1d function which histograms the particles time coordinates $t$ and normalizates the histogram so that the resulting PDF has units of current.

In [ ]:
plot_current_profile(beam, 1, units);

In [ ]:
# Cleanup
os.remove('rad.gaussian.gpt.out.txt')
os.remove('rad.gaussian.astra.out.txt')
os.remove('rad.gaussian.openPMD.out.h5')

# Running Distgen with driver functions
The main in python driver function for distgen is distgen.drivers.run_distgen().  The function creates its own Generator object and returns a beam object.  Inputs can either be an inputfile or a parsed distgen supported input dictionary.  Here the function is run with an input dictionary created above.

In [ ]:
from distgen.drivers import run_distgen
beam = run_distgen(inputs=p, verbose=1)

Below the function is called with 'inputs' pointing to an input file.  The function can also take  accept a flattened settings dict, which updates inputs:

In [ ]:
new_settings = {'beam:params:MTE:value':0.1}
beam = run_distgen(settings=new_settings, inputs=input_file, verbose=1)

Finally, is also possible to run distgen as an executible script using /bin/DistGen.py
import subprocess

In [ ]:
import subprocess
output=subprocess.check_output(['python','../bin/DistGen.py','-f','data/rad.gaussian.in.json','-v','2'])
output=str(output.decode('utf-8')).split("\n")
for line in output:
    print(line)

# Scaling/Rotating/Shifting Coordinates

The distgen module allows the user with several options for applying coordinate transformations.  Some of the basic operations include: shifting, scaling, and rotating coordinates.  See the transform.ipynb for a detailed description of this functionality. 

# Cathode Start

Currently Distgen supports a single emission model.  Particles emitted from a cathode are assumed to have a Maxwell-Boltzmann distribution for their total mometnum $p$ parameterized by an energy scale denoted $MTE$: 

$\rho_p(p) = \left(\frac{1}{2\pi mMTE}\right)^{3/2} 4\pi p^2 \exp\left(-\frac{p^2}{2m MTE}\right)$, with $\int\rho_p(p)dp = 1$.

Assuming spherical symmetry, it is easy to show the corresponding PDF for the components of the momenta is given by:

$\rho(p_x,p_y,p_z) = \left(\frac{1}{2\pi mMTE}\right)^{3/2}\exp\left(-\frac{p_x^2+p_y^2+p_z^2}{2m MTE}\right)$, with $\int\rho(p_x,p_y,p_z) d^3p=1$.

From the equipartition theorem the average energy $\frac{3}{2}MTE$ is divided equally among each direction and thus the energy scale $MTE$ can be identified as the Mean Transverse Energy: $MTE=\frac{\langle p_x^2\rangle}{2m} + \frac{\langle p_y^2\rangle}{2m}$.  

This expression can be used to relate the MTE to the initial cathode emittance:

$\epsilon_{n,x} = \sigma_{x}\sqrt{ \frac{MTE}{mc^2} }$

assuming there is no distinction between the $p_x$ and $p_y$ distributions inherent to the emission process.  To account for the fact that particles with $p_z<0$ are not emitted from the cathode, distgen takes $p_z = |p_z|$. 

So far, all of the examples shown above describes beam distributions emitted from a cathode.  The input required for cathode start is shown below:

In [ ]:
from distgen import Generator
input_file = "data/rad.gaussian.in.json"

gen = Generator(input=input_file,verbose=1)
print('beam:start_type:',gen.input['beam']['start_type'], ', beam:params:MTE:', gen.input['beam']['params']['MTE'])

Running the Generator.beam() function with verbose >=1 will show the stats for generation of the momenta. Note the absolute value of the $p_z$ components is taken.

In [ ]:
beam = gen.beam()

The time coordinate in this case effectively respresents the emission time of the particle from the cathode, and thus the time distribution should be set by the user.  The plots below show the corresponding horizontal and longitudinal phase spaces:

In [ ]:
fig, ax = plt.subplots(1, 2, sharex='col',constrained_layout=True)
plot_2d(beam,1,"x",units["x"],"px",units["px"],'scatter_hist2d',nbins=100,ax=ax[0]);
ax[0].set_title('x-Px Phase Space')

plot_2d(beam,1,"t",units["t"],"pz",units["pz"],'scatter_hist2d',nbins=100,ax=ax[1]);
ax[1].set_title('t-Pz Phase Space');

# Time Start

Distgen also allows the user to start a beam at an arbitrary time.  In this case, distgen will ignore any user distribution settings for the time coordinate of the beam particles.  The example below shows a semi-realistic situation for a 1 GeV Gaussian (all dimensions) beam after an RF energy spread is added to longitudinal distribution.

In [ ]:
filename = "data/gaussian.in.json"
from distgen.physical_constants import unit_registry, pi, c

gen = Generator(filename, verbose=0) 

beta_params = {'beta': {'value':12.5, 'units':'m'},
               'alpha':{'value':-1, 'units':''},
               'eps':  {'value':2,'units':'nm'}}

f = 1.3*unit_registry('GHz')
w = 2*pi*f/c

linac_params = {'amplitude':{'value':1,'units':'GeV/c'},
               'omega':{'value':w.magnitude, 'units':str(w.units)},
               'phase':{'value':-1, 'units':'deg'}}

gen.input['transforms']={
    'pZ-boost':  {'type':'cosine', 'variables':'z:pz','params':linac_params},
    'setXtwiss': {'type':'set_twiss', 'variables':'x', 'params':beta_params},
    'setYtwiss': {'type':'set_twiss', 'variables':'y', 'params':beta_params},
}

beam = gen.beam()

fig1, ax1 = plt.subplots(1, 3, constrained_layout=True)
plot_2d(beam,fig1,"x",units["x"],"y",units["y"],'scatter_hist2d',nbins=100,ax=ax1[0],axis='equal');
plot_2d(beam,fig1,"x",units["x"],"thetax",units["thetax"],'scatter_hist2d',nbins=100,ax=ax1[1]);
plot_2d(beam,fig1,"z",units["z"],"pz",'GeV/c','scatter_hist2d',nbins=100,ax=ax1[2]);